In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/falaaharifkhan/Documents/fairness-variance


## Import dependencies

In [4]:
import os
import pandas as pd
from IPython.display import display
from tqdm import tqdm

from configs import config
from configs.constants import ModelSetting
from utils.analyzers.subgroups_variance_analyzer import SubgroupsVarianceAnalyzer
from utils.common_helpers import create_tuned_base_model, save_metrics_to_file
from utils.custom_classes.data_loader import ACSEmploymentDataset
from utils.custom_classes.generic_pipeline import GenericPipeline
from utils.analyzers.subgroups_statistical_bias_analyzer import SubgroupsStatisticalBiasAnalyzer

## Configs

In [5]:
STATE = config.DATASET_CONFIG['state']
YEAR = config.DATASET_CONFIG['year']
DATASET_NAME = f"Folktables_{STATE}_{YEAR}"
EXPERIMENT_NAME = 'Hypothesis_Space'

seed_lst = [101, 111, 201, 221, 311, 333, 410, 555, 666, 851, 999, 1010]
SEX_priv = RACE_priv = str(1)
N_ESTIMATORS = 200
SENSITIVE_ATTRIBUTES = ['SEX','RAC1P']
PRIV_VALUES = [SEX_priv, RACE_priv]
TUNED_PARAMS_FILE_PATH = os.path.join('results', 'models_tuning', 'tuning_results_Folktables_GA_2018_20230117__023240.csv')

## Models tuned hyper-parameters

In [6]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

,Unnamed: 0,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,0,Folktables_GA_2018,LogisticRegression,0.8195,0.8203,"{'C': 1, 'max_iter': 150, 'penalty': 'l2', 'so..."
1,1,Folktables_GA_2018,DecisionTreeClassifier,0.8304,0.8313,"{'criterion': 'gini', 'max_depth': 10, 'max_fe..."
2,2,Folktables_GA_2018,RandomForestClassifier,0.8354,0.8357,"{'max_depth': 10, 'max_features': 0.6, 'min_sa..."
3,3,Folktables_GA_2018,XGBClassifier,0.8371,0.8375,"{'lambda': 1, 'learning_rate': 0.1, 'max_depth..."
4,4,Folktables_GA_2018,KNeighborsClassifier,0.8136,0.8137,"{'metric': 'minkowski', 'n_neighbors': 25, 'we..."
5,5,Folktables_GA_2018,MLPClassifier,0.8345,0.8353,"{'activation': 'logistic', 'hidden_layer_sizes..."


## Load dataset

In [7]:
dataset = ACSEmploymentDataset(state=[STATE], year=YEAR, root_dir=os.path.join('data'), with_nulls=False, subsample=20000)
dataset.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
36639,1,4,0,1,2,1,2,2,2,1,1,1,1,1,18,66
60712,1,4,0,1,2,1,1,1,2,1,1,0,1,1,16,63
60089,1,4,0,1,2,1,2,2,2,2,1,0,1,4,19,61
89170,1,2,0,1,2,1,1,2,2,1,2,0,1,1,20,59
24166,1,4,0,3,2,1,1,1,2,2,1,1,1,4,16,63


## Run experiments

In [8]:
def create_base_pipeline(dataset, sensitive_attributes, priv_values, model_seed):
    base_pipeline = GenericPipeline(dataset, sensitive_attributes, priv_values)
    _ = base_pipeline.create_preprocessed_train_test_split(dataset, config.TEST_SET_FRACTION, seed=model_seed)

    print('\nProtected groups splits:')
    for g in base_pipeline.test_groups.keys():
        print(g, base_pipeline.test_groups[g].shape)

    return base_pipeline


def get_model_metrics(base_model, n_estimators, dataset, sensitive_attributes, priv_values, model_seed_lst,
                      dataset_name, base_model_name, exp_num=1):
    
    results_over_all_runs = pd.DataFrame({})
    
    for model_seed in model_seed_lst:
        base_pipeline = create_base_pipeline(dataset, sensitive_attributes, priv_values, model_seed)

        # Compute variance metrics for subgroups
        save_results = True
        stability_fairness_analyzer = SubgroupsVarianceAnalyzer(ModelSetting.BATCH, n_estimators, base_model, base_model_name,
                                                                base_pipeline.X_train_val, base_pipeline.y_train_val,
                                                                base_pipeline.X_test, base_pipeline.y_test,
                                                                base_pipeline.sensitive_attributes, base_pipeline.priv_values, base_pipeline.test_groups,
                                                                base_pipeline.target, dataset_name)

        y_preds, variance_metrics_df = stability_fairness_analyzer.compute_metrics(save_results=save_results,
                                                                                   result_filename=None,
                                                                                   save_dir_path=None,
                                                                                   make_plots=False)

        # Compute bias metrics for subgroups
        bias_analyzer = SubgroupsStatisticalBiasAnalyzer(base_pipeline.X_test, base_pipeline.y_test,
                                                         base_pipeline.sensitive_attributes, base_pipeline.priv_values,
                                                         base_pipeline.test_groups)
        dtc_res = bias_analyzer.compute_subgroups_metrics(y_preds,
                                                          save_results=False,
                                                          result_filename=None,
                                                          save_dir_path=None)
        bias_metrics_df = pd.DataFrame(dtc_res)

        metrics_df = pd.concat([variance_metrics_df, bias_metrics_df])
        metrics_df['SEED'] = [model_seed]*len(metrics_df)
        
        if len(results_over_all_runs) == 0:
            print("saving first")
            results_over_all_runs = metrics_df.deep_copy=True
        else:
            results_over_all_runs = pd.concat([results_over_all_runs, metrics_df], ignore_index=True, copy=True)

    
    # Save metrics    
    result_filename = f'{EXPERIMENT_NAME}_Metrics_{dataset_name}_Experiment_{exp_num}_{base_model_name}'
    save_dir_path = os.path.join('results', 'hypothesis_space')
    save_metrics_to_file(results_over_all_runs, result_filename, save_dir_path)

    return results_over_all_runs


In [9]:
def run_experiment(exp_num: int, model_seed_lst: int):
    """
    Find variance and bias metrics for each model in config.MODELS_CONFIG.
    Save results in results/config.MODELS_CONFIG folder.

    :param exp_num: the number of experiment; is used to name the result file with metrics
    """
    all_results = {}
    for model_idx in tqdm(range(len(config.MODELS_CONFIG))):
        print('#' * 30, f' [Experiment {exp_num}] Analyze {config.MODELS_CONFIG[model_idx]["model_name"]} ', '#' * 30)
        #model_seed += 1
        try:
            base_model = create_tuned_base_model(config.MODELS_CONFIG[model_idx]['model'],
                                                 config.MODELS_CONFIG[model_idx]['model_name'],
                                                 models_tuned_params_df)
            results_df = get_model_metrics(base_model, N_ESTIMATORS, dataset, SENSITIVE_ATTRIBUTES, PRIV_VALUES,
                                           model_seed_lst=model_seed_lst,
                                           dataset_name=DATASET_NAME,
                                           base_model_name=config.MODELS_CONFIG[model_idx]['model_name'],
                                           exp_num=exp_num)
            print(f'\n[Experiment {exp_num}] Metrics confusion matrix:')
            display(results_df)
            all_results[config.MODELS_CONFIG[model_idx]['model_name']] = results_df
        except Exception as err:
            print(f'ERROR with {config.MODELS_CONFIG[model_idx]["model_name"]}: ', err)

        print('\n\n\n')
    return all_results


### Experiment 1

In [10]:
# TOD: add dataset as a parameter
all_results_df = run_experiment(exp_num=1, model_seed_lst=seed_lst)

  0%|                                                     | 0/6 [00:00<?, ?it/s]2023-01-17 01:25:46 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 1 / 200


##############################  [Experiment 1] Analyze LogisticRegression  ##############################
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)

Protected groups splits:
SEX_RAC1P_priv (1269, 16)
SEX_RAC1P_dis (729, 16)
SEX_priv (1916, 16)
SEX_dis (2084, 16)
RAC1P_priv (2624, 16)
RAC1P_dis (1376, 16)


2023-01-17 01:25:47 abstract_overall_variance_analyzer.py INFO    : Classifier 1 / 200 was tested
2023-01-17 01:25:47 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 2 / 200
2023-01-17 01:25:47 abstract_overall_variance_analyzer.py INFO    : Classifier 2 / 200 was tested
2023-01-17 01:25:47 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 3 / 200
2023-01-17 01:25:47 abstract_overall_variance_analyzer.py INFO    : Classifier 3 / 200 was tested
2023-01-17 01:25:47 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 4 / 200
2023-01-17 01:25:48 abstract_overall_variance_analyzer.py INFO    : Classifier 4 / 200 was tested
2023-01-17 01:25:48 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 5 / 200
2023-01-17 01:25:48 abstract_overall_variance_analyzer.py INFO    : Classifier 5 / 200 was tested
2023-01-17 01:25:48 abstract_overall_variance_analyzer.py INFO    : Start testing of classifie

2023-01-17 01:25:59 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 42 / 200
2023-01-17 01:25:59 abstract_overall_variance_analyzer.py INFO    : Classifier 42 / 200 was tested
2023-01-17 01:25:59 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 43 / 200
2023-01-17 01:25:59 abstract_overall_variance_analyzer.py INFO    : Classifier 43 / 200 was tested
2023-01-17 01:25:59 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 44 / 200
2023-01-17 01:25:59 abstract_overall_variance_analyzer.py INFO    : Classifier 44 / 200 was tested
2023-01-17 01:25:59 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 45 / 200
2023-01-17 01:26:00 abstract_overall_variance_analyzer.py INFO    : Classifier 45 / 200 was tested
2023-01-17 01:26:00 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 46 / 200
2023-01-17 01:26:00 abstract_overall_variance_analyzer.py INFO    : Classifier 

2023-01-17 01:26:10 abstract_overall_variance_analyzer.py INFO    : Classifier 82 / 200 was tested
2023-01-17 01:26:10 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 83 / 200
2023-01-17 01:26:10 abstract_overall_variance_analyzer.py INFO    : Classifier 83 / 200 was tested
2023-01-17 01:26:10 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 84 / 200
2023-01-17 01:26:11 abstract_overall_variance_analyzer.py INFO    : Classifier 84 / 200 was tested
2023-01-17 01:26:11 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 85 / 200
2023-01-17 01:26:11 abstract_overall_variance_analyzer.py INFO    : Classifier 85 / 200 was tested
2023-01-17 01:26:11 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 86 / 200
2023-01-17 01:26:11 abstract_overall_variance_analyzer.py INFO    : Classifier 86 / 200 was tested
2023-01-17 01:26:11 abstract_overall_variance_analyzer.py INFO    : Start testing of 

...

2023-01-17 02:50:32 abstract_overall_variance_analyzer.py INFO    : Classifier 122 / 200 was tested
2023-01-17 02:50:32 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 123 / 200
2023-01-17 02:50:57 abstract_overall_variance_analyzer.py INFO    : Classifier 123 / 200 was tested
2023-01-17 02:50:57 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 124 / 200
2023-01-17 02:51:24 abstract_overall_variance_analyzer.py INFO    : Classifier 124 / 200 was tested
2023-01-17 02:51:24 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 125 / 200
2023-01-17 02:51:51 abstract_overall_variance_analyzer.py INFO    : Classifier 125 / 200 was tested
2023-01-17 02:51:51 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 126 / 200
2023-01-17 02:52:17 abstract_overall_variance_analyzer.py INFO    : Classifier 126 / 200 was tested
2023-01-17 02:52:17 abstract_overall_variance_analyzer.py INFO    : Start te

2023-01-17 03:07:31 abstract_overall_variance_analyzer.py INFO    : Classifier 162 / 200 was tested
2023-01-17 03:07:31 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 163 / 200
2023-01-17 03:07:54 abstract_overall_variance_analyzer.py INFO    : Classifier 163 / 200 was tested
2023-01-17 03:07:54 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 164 / 200
2023-01-17 03:08:19 abstract_overall_variance_analyzer.py INFO    : Classifier 164 / 200 was tested
2023-01-17 03:08:19 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 165 / 200
2023-01-17 03:08:43 abstract_overall_variance_analyzer.py INFO    : Classifier 165 / 200 was tested
2023-01-17 03:08:43 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 166 / 200
2023-01-17 03:09:07 abstract_overall_variance_analyzer.py INFO    : Classifier 166 / 200 was tested
2023-01-17 03:09:07 abstract_overall_variance_analyzer.py INFO    : Start te



##############################  Stability metrics  ##############################
General Ensemble Accuracy: 0.8308
Mean: 0.5588
Std: 0.0629
IQR: 0.0813
Entropy: 0.1138
Jitter: 0.0707
Per sample accuracy: 0.8245
Label stability: 0.9024




100%|█████████████████████████████████████████| 6/6 [1:59:11<00:00, 1191.85s/it]

ERROR with MLPClassifier:  expected str, bytes or os.PathLike object, not NoneType






### Experiment 2

In [11]:
# run_experiment(exp_num=2, model_seed=200)